# Title
[]()

In [2]:

import sys
sys.path.append(r"/home/silvhua/custom_python")
sys.path.append(r"/home/silvhua/repositories/notion/src")
from silvhua import *

In [ ]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Load DataFrame

In [3]:
filename = 'notion_df.sav'
path = '/home/silvhua/repositories/notion/data/'
df = loadpickle(filename, path)
df

	Time completed: 2023-12-31 13:32:43.724703
Dataframe shape:  (1353, 15)
DataFrame columns: ['Name', 'created_time', 'Task Tag', 'Flag', 'Elapsed', 'Task Name', 'Task Project tags', 'Task Parent-task', 'Task Sub-tasks', 'Task Project name', 'start time', 'end time', 'Roadmap Item', 'url', 'id']


,Name,created_time,Task Tag,Flag,Elapsed,Task Name,Task Project tags,Task Parent-task,Task Sub-tasks,Task Project name,start time,end time,Roadmap Item,url,id
0,2023-09-01 09:48,2023-09-01 09:48:00-07:00,[life],None,0.783333,CloudWolf CCP 75% of content,[tech ed],[None],[None],AWS Certified Cloud Practitioner certification,09:48,10:35,[],https://www.notion.so/2023-09-01-09-48-a0526a3...,851
1,2023-09-01 10:55,2023-09-01 10:55:00-07:00,[life],None,0.333333,create social media content showing my expertise,[life],[None],"[[3d5b9f63-419e-4d4c-904a-f0d66f5951a5, e54598...",career positioning,10:35,10:55,[],https://www.notion.so/2023-09-01-10-55-68ed3f5...,850
2,;2023-09-01 10:57,2023-09-01 10:57:00-07:00,[Ginkgo],None,0.766667,Look into WriteSonic AI social media tool (Ruo...,[Ginkgo work],[None],[None],knowledge library social media,10:57,11:43,[],https://www.notion.so/2023-09-01-10-57-ce0b078...,849
3,2023-09-01 11:59,2023-09-01 11:59:00-07:00,[Ginkgo],None,0.766667,Ginkgo sprint meeting,[Ginkgo work],[None],[None],Ginkgo Admin,12:00,12:46,[],https://www.notion.so/2023-09-01-11-59-3e167f5...,848
4,2023-09-01 13:30,2023-09-01 13:30:00-07:00,[life],None,1.416667,CloudWolf CCP 75% of content,[tech ed],[None],[None],AWS Certified Cloud Practitioner certification,13:30,14:55,[],https://www.notion.so/2023-09-01-13-30-33447cd...,847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1348,2023-12-30 19:55,2023-12-30 19:55:00-08:00,[life],None,0.583333,learn Solara & Ploomber,"[life, tech ed, admin]",[None],[None],Notion analytics,19:55,20:30,[],https://www.notion.so/2023-12-30-19-55-f3a69e5...,276
1349,2023-12-30 19:55,2023-12-30 19:55:00-08:00,[life],None,0.583333,learn Solara & Ploomber,"[life, tech ed, admin]",[None],[None],Notion analytics,19:55,20:30,[],https://www.notion.so/2023-12-30-19-55-f3a69e5...,1
1350,2023-12-30 20:30,2023-12-30 20:30:00-08:00,[life],None,3.5,Wrangle time tracking data,"[life, tech ed, admin]",[None],[None],Notion analytics,20:30,24:00,[],https://www.notion.so/2023-12-30-20-30-b015b92...,278
1351,2023-12-30 20:30,2023-12-30 20:30:00-08:00,[life],None,3.5,Wrangle time tracking data,"[life, tech ed, admin]",[None],[None],Notion analytics,20:30,24:00,[],https://www.notion.so/2023-12-30-20-30-b015b92...,0


# Process DataFrame

In [4]:
from wrangling import *

def classify_unbilled(df, column='Flag', flag_name='do not bill'):

    def parse_flag(row, flag_name=flag_name):
        if row[column]:
            for flag in row[column]:
                if flag['name'] == flag_name:
                    return True
                else:
                    return False
        else:
            return False

    classified_df = df.copy()
    classified_df['Unbilled'] = classified_df.apply(parse_flag, axis=1)
    return classified_df

    
def classify_projects(df, column='Task Project name', tag_column='Task Project tags'):
    def classify(row):
        tag_set = set(row[tag_column]) if len(row[tag_column]) > 0 else {row[tag_column]}
        if 'ginkgo' in row[column].lower():
            row['Category'] = 'Ginkgo'
        elif row[column].lower() in ['ghl chatbot', 'tech business']:
            row['Category'] = 'tech business'
            if row[column].lower() == 'tech business':
                row['Unbilled'] = True
        elif tag_set.intersection({'portfolio', 'tech ed'}):
            row['Category'] = 'tech career development'
            row['Unbilled'] = True
        elif 'defy time fitness' in [tag.lower() for tag in row[tag_column]]:
            row['Category'] = 'personal training work'
            if 'defy time fitness' not in row[column].lower():
                row['Unbilled'] = True
        elif row[column].lower() == 'career positioning':
            row['Category'] = 'career positioning'
            row['Unbilled'] = True
        else:
            row['Category'] = 'Other'
            row['Unbilled'] = True

        return row
    
    classified_df = classify_unbilled(df).apply(lambda x: classify(x), axis=1)
    unique_categories = classified_df['Category'].unique()
    print(f'There are {len(unique_categories)} categories: {[category for category in unique_categories]}')
    return classified_df

classified_df = classify_projects(df)
classified_df

There are 6 categories: ['tech career development', 'career positioning', 'Other', 'Ginkgo', 'personal training work', 'tech business']


,Name,created_time,Task Tag,Flag,Elapsed,Task Name,Task Project tags,Task Parent-task,Task Sub-tasks,Task Project name,start time,end time,Roadmap Item,url,id,Unbilled,Category
0,2023-09-01 09:48,2023-09-01 09:48:00-07:00,[life],None,0.783333,CloudWolf CCP 75% of content,[tech ed],[None],[None],AWS Certified Cloud Practitioner certification,09:48,10:35,[],https://www.notion.so/2023-09-01-09-48-a0526a3...,851,True,tech career development
1,2023-09-01 10:55,2023-09-01 10:55:00-07:00,[life],None,0.333333,create social media content showing my expertise,[life],[None],"[[3d5b9f63-419e-4d4c-904a-f0d66f5951a5, e54598...",career positioning,10:35,10:55,[],https://www.notion.so/2023-09-01-10-55-68ed3f5...,850,True,career positioning
2,;2023-09-01 10:57,2023-09-01 10:57:00-07:00,[Ginkgo],None,0.766667,Look into WriteSonic AI social media tool (Ruo...,[Ginkgo work],[None],[None],knowledge library social media,10:57,11:43,[],https://www.notion.so/2023-09-01-10-57-ce0b078...,849,True,Other
3,2023-09-01 11:59,2023-09-01 11:59:00-07:00,[Ginkgo],None,0.766667,Ginkgo sprint meeting,[Ginkgo work],[None],[None],Ginkgo Admin,12:00,12:46,[],https://www.notion.so/2023-09-01-11-59-3e167f5...,848,False,Ginkgo
4,2023-09-01 13:30,2023-09-01 13:30:00-07:00,[life],None,1.416667,CloudWolf CCP 75% of content,[tech ed],[None],[None],AWS Certified Cloud Practitioner certification,13:30,14:55,[],https://www.notion.so/2023-09-01-13-30-33447cd...,847,True,tech career development
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1348,2023-12-30 19:55,2023-12-30 19:55:00-08:00,[life],None,0.583333,learn Solara & Ploomber,"[life, tech ed, admin]",[None],[None],Notion analytics,19:55,20:30,[],https://www.notion.so/2023-12-30-19-55-f3a69e5...,276,True,tech career development
1349,2023-12-30 19:55,2023-12-30 19:55:00-08:00,[life],None,0.583333,learn Solara & Ploomber,"[life, tech ed, admin]",[None],[None],Notion analytics,19:55,20:30,[],https://www.notion.so/2023-12-30-19-55-f3a69e5...,1,True,tech career development
1350,2023-12-30 20:30,2023-12-30 20:30:00-08:00,[life],None,3.500000,Wrangle time tracking data,"[life, tech ed, admin]",[None],[None],Notion analytics,20:30,24:00,[],https://www.notion.so/2023-12-30-20-30-b015b92...,278,True,tech career development
1351,2023-12-30 20:30,2023-12-30 20:30:00-08:00,[life],None,3.500000,Wrangle time tracking data,"[life, tech ed, admin]",[None],[None],Notion analytics,20:30,24:00,[],https://www.notion.so/2023-12-30-20-30-b015b92...,0,True,tech career development


In [46]:
filters = {
    'Category': ['other']
}
filter_df_all_conditions(classified_df, filters)

Results where ALL conditions are met:
	DataFrame indices: [2, 7, 8, 11, 14, 15, 26, 29, 30, 33, 34, 35, 36, 38, 39, 40, 41, 45, 46, 48, 49, 50, 51, 55, 57, 60, 63, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 81, 98, 99, 103, 104, 106, 114, 115, 117, 118, 126, 128, 132, 133, 135, 136, 137, 139, 140, 141, 142, 145, 150, 151, 153, 154, 157, 158, 163, 165, 166, 169, 178, 179, 180, 184, 185, 213, 214, 216, 217, 218, 220, 221, 222, 224, 225, 226, 227, 228, 230, 231, 232, 233, 235, 237, 239, 240, 241, 242, 243, 244, 245, 246, 248, 249, 250, 252, 253, 254, 255, 256, 257, 259, 260, 261, 262, 266, 269, 270, 271, 273, 274, 275, 278, 279, 280, 283, 284, 288, 290, 291, 293, 294, 296, 297, 298, 300, 301, 302, 303, 306, 308, 309, 311, 314, 316, 317, 318, 319, 322, 323, 325, 326, 327, 328, 329, 330, 331, 332, 333, 335, 338, 339, 341, 342, 344, 346, 348, 349, 350, 351, 352, 357, 358, 359, 361, 362, 363, 364, 365, 366, 367, 368, 370, 371, 373, 384, 390, 391, 395, 397, 398, 399, 400, 401, 402, 404, 405, 406,

,Name,created_time,Task Tag,Flag,Elapsed,Task Name,Task Project tags,Task Parent-task,Task Sub-tasks,Task Project name,start time,end time,Roadmap Item,url,id,Unbilled,Category
2,;2023-09-01 10:57,2023-09-01 10:57:00-07:00,[Ginkgo],None,0.766667,Look into WriteSonic AI social media tool (Ruo...,[Ginkgo work],[None],[None],knowledge library social media,10:57,11:43,[],https://www.notion.so/2023-09-01-10-57-ce0b078...,849,True,Other
7,2023-09-01 17:05,2023-09-01 17:05:00-07:00,[life],None,0.633333,personal dev ops,[admin],[None],[None],Admin,16:40,17:18,[],https://www.notion.so/2023-09-01-17-05-594fb17...,844,True,Other
8,2023-09-02 00:59,2023-09-02 00:58:00-07:00,[life],None,1.650000,personal dev ops,[admin],[None],[None],Admin,23:20,24:59,[],https://www.notion.so/2023-09-02-00-59-de0ce1f...,843,True,Other
11,2023-09-03 14:11,2023-09-03 14:11:00-07:00,[Ginkgo],None,2.866667,update knowledge library tagging system; add c...,[Ginkgo work],[None],[None],Refine knowledge library pipeline,14:14,17:06,[],https://www.notion.so/2023-09-03-14-11-9d36a4d...,840,True,Other
14,2023-09-04 17:05,2023-09-04 17:05:00-07:00,[Ginkgo],None,0.550000,get articles from Examine.com,[Ginkgo work],[None],[None],populate knowledge library,17:05,17:38,[],https://www.notion.so/2023-09-04-17-05-67059ad...,837,True,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,2023-12-28,2023-12-28 11:10:00-08:00,[life],None,0.916667,counselling session,[life],[None],[None],Inner work,9:00,9:55,[],https://www.notion.so/2023-12-28-546726b1599e4...,250,True,Other
1314,2023-12-29 09:07s,2023-12-29 09:07:00-08:00,[],None,0.083333,invoicing Ginkgo,[admin],[None],[None],Admin,9:07,9:12,[],https://www.notion.so/2023-12-29-09-07s-be011f...,18,True,Other
1315,2023-12-29 09:07s,2023-12-29 09:07:00-08:00,[],None,0.083333,invoicing Ginkgo,[admin],[None],[None],Admin,9:07,9:12,[],https://www.notion.so/2023-12-29-09-07s-be011f...,260,True,Other
1324,2023-12-29 20:40,2023-12-29 20:40:00-08:00,[life],None,0.150000,personal sprint planning,[admin],[None],[None],Admin,20:40,20:49,[],https://www.notion.so/2023-12-29-20-40-eaa738e...,13,True,Other


# Iteration 1

In [31]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_aggregate(df, groupby, columns=None, classification=None, label=1, barmode='stack', n_columns=1, height=150, y_order=None):
    """
    Use Plotly to plot multiple histograms using the specified columns of a dataframe.
    https://plotly.com/python/histograms/#specify-aggregation-function

    Arguments:
    - df: Dataframe.
    - columns (optional): Columns of dataframe on which to create the histogram. If blank, all numeric data will be plotted.
    - classification (optional): Provide name of colum containing binary classification values 0 and 1. 
        Data points classified as 1 will be in red.
    - label (optional): Label of classification column. Default is 1.
    - barmode ('stack', 'group', or 'overlay'; optional): How the different will be shown. Default is 'stack'.
    - n_columns (optional): Number of columns in the figure. Default is 2.
    - height (optional): Height of each subplot in pixels. Default is 150.
    - y_order (optional): List of values to specify the order of the y axis. Default is None.

    """
    if columns == None:
        columns = df.dtypes[df.dtypes != 'object'].index.tolist()
    n_rows = (len(columns)-1) // n_columns + 1
    fig = make_subplots(rows=n_rows, cols=n_columns, subplot_titles=columns)
    for i, feature in enumerate(columns):
        if classification:
            zero = df.sort_values(feature)[df.sort_values(feature)[classification] != label]
            one = df.sort_values(feature)[df.sort_values(feature)[classification] == label]
            fig.add_trace(go.Histogram(y=zero[feature],
                marker_color='blue',
                orientation='h',
                opacity=0.5), 
                row=i//n_columns+1, col=i % n_columns + 1
                )
            fig.add_trace(go.Histogram(y=one[feature],
                marker_color='red',
                orientation='h',
                opacity=0.5),
                row=i//n_columns+1, col=i % n_columns + 1)
        else:
            fig.add_trace(go.Histogram(
                histfunc="sum", x=df[feature], y=df[groupby],
                orientation='h'
                ), 
            row=i//n_columns+1, col=i % n_columns + 1)
    
    if classification:
        title = f'Observations with {classification} of value {label} are indicated in red'
    else:
        title = 'Value counts'
    fig.update_layout(
        showlegend=False,
        height=(n_rows+1)*height,
        barmode=barmode,
        # bargap=0.1,
        title=title,
        title_x=0.5,
        title_xanchor='center',
        # title_y=0.1,
        title_yanchor='top'
    )
    fig.update_xaxes(title=dict(
        standoff=0,
        ),
        title_text='number of observations',
        row=n_rows
    )
    if y_order:
        print(f'y_order: {y_order}')
        fig.update_yaxes(categoryorder='array', categoryarray=y_order)
    fig.show()


def plot_by_category(df, category_column='Category', sort_column='Elapsed'):
    classified_df = classify_projects(df)
    aggregate_df = classified_df[[sort_column, category_column]].groupby(
        category_column
        ).sum().sort_values(by=[sort_column], ascending=True)
    index_list = aggregate_df.index.tolist()
    print(index_list)

    plot_aggregate(
        classified_df, 
        groupby=category_column,
        columns=[sort_column],
        # classification='Unbilled',
        # label=True,
        y_order=index_list
    )
    return aggregate_df

plot_by_category(df)

There are 6 categories: ['tech career development', 'career positioning', 'Other', 'Ginkgo', 'personal training work', 'tech business']
['personal training work', 'career positioning', 'Ginkgo', 'tech business', 'Other', 'tech career development']
y_order: ['personal training work', 'career positioning', 'Ginkgo', 'tech business', 'Other', 'tech career development']


,Elapsed
Category,
personal training work,31.666667
career positioning,44.083333
Ginkgo,99.900000
tech business,148.033333
Other,301.183333
tech career development,370.350000


## 1.1

In [39]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_int_hist(
    df, groupby, columns=None, classification=None, label=1, 
    agg='sum',
    barmode='stack', n_columns=1, height=150, y_order=None
    ):
    """
    Use Plotly to plot multiple histograms using the specified columns of a dataframe.
    https://plotly.com/python/histograms/#specify-aggregation-function

    Arguments:
    - df: Dataframe.
    - columns (optional): Columns of dataframe on which to create the histogram. If blank, all numeric data will be plotted.
    - classification (optional): Provide name of colum containing binary classification values 0 and 1. 
        Data points classified as 1 will be in red.
    - label (optional): Label of classification column. Default is 1.
    - barmode ('stack', 'group', or 'overlay'; optional): How the different will be shown. Default is 'stack'.
    - n_columns (optional): Number of columns in the figure. Default is 2.
    - height (optional): Height of each subplot in pixels. Default is 150.
    - y_order (optional): List of values to specify the order of the y axis. Default is None.

    """
    if columns == None:
        columns = df.dtypes[df.dtypes != 'object'].index.tolist()
    n_rows = (len(columns)-1) // n_columns + 1
    fig = make_subplots(rows=n_rows, cols=n_columns, subplot_titles=columns)
    for i, feature in enumerate(columns):
        if classification:
            zero = df.sort_values(feature)[df.sort_values(feature)[classification] != label]
            one = df.sort_values(feature)[df.sort_values(feature)[classification] == label]
            fig.add_trace(go.Histogram(y=zero[groupby], x=zero[feature],
                histfunc=agg, 
                marker_color='blue',
                orientation='h',
                opacity=0.5), 
                row=i//n_columns+1, col=i % n_columns + 1
                )
            fig.add_trace(go.Histogram(y=one[groupby], x=one[feature],
                histfunc=agg, 
                marker_color='red',
                orientation='h',
                opacity=0.5),
                row=i//n_columns+1, col=i % n_columns + 1)
        else:
            fig.add_trace(go.Histogram(
                histfunc=agg, x=df[feature], y=df[groupby],
                orientation='h'
                ), 
            row=i//n_columns+1, col=i % n_columns + 1)
    
    if classification:
        title = f'Observations with {classification} of value {label} are indicated in red'
    else:
        title = 'Value counts'
    fig.update_layout(
        showlegend=False,
        height=(n_rows+1)*height,
        barmode=barmode,
        # bargap=0.1,
        title=title,
        title_x=0.5,
        title_xanchor='center',
        # title_y=0.1,
        title_yanchor='top'
    )
    fig.update_xaxes(title=dict(
        standoff=0,
        ),
        title_text=f'{agg}'.upper(),
        row=n_rows
    )
    if y_order:
        print(f'y_order: {y_order}')
        fig.update_yaxes(categoryorder='array', categoryarray=y_order)
    fig.show()


def plot_by_category(
    df, category_column='Category', classification='Unbilled', label=True,
    agg='sum',
    sort_column='Elapsed'
    ):
    classified_df = classify_projects(df)
    aggregate_df = classified_df[[sort_column, category_column]].groupby(
        category_column
        ).sum().sort_values(by=[sort_column], ascending=True)
    index_list = aggregate_df.index.tolist()
    print(index_list)

    plot_int_hist(
        classified_df, 
        groupby=category_column,
        columns=[sort_column],
        classification=classification,
        agg=agg,
        label=label,
        y_order=index_list
    )
    return aggregate_df

plot_by_category(df, classification='Unbilled')

There are 6 categories: ['tech career development', 'career positioning', 'Other', 'Ginkgo', 'personal training work', 'tech business']
['personal training work', 'career positioning', 'Ginkgo', 'tech business', 'Other', 'tech career development']
y_order: ['personal training work', 'career positioning', 'Ginkgo', 'tech business', 'Other', 'tech career development']


,Elapsed
Category,
personal training work,31.666667
career positioning,44.083333
Ginkgo,99.900000
tech business,148.033333
Other,301.183333
tech career development,370.350000


## 1.2

In [43]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_int_hist(
    df, groupby, columns=None, classification=None, label=1, 
    agg='sum',
    barmode='stack', n_columns=1, height=150, y_order=False
    ):
    """
    Use Plotly to plot multiple histograms using the specified columns of a dataframe.
    https://plotly.com/python/histograms/#specify-aggregation-function

    Arguments:
    - df: Dataframe.
    - columns (optional): Columns of dataframe on which to create the histogram. If blank, all numeric data will be plotted.
    - classification (optional): Provide name of colum containing binary classification values 0 and 1. 
        Data points classified as 1 will be in red.
    - label (optional): Label of classification column. Default is 1.
    - barmode ('stack', 'group', or 'overlay'; optional): How the different will be shown. Default is 'stack'.
    - n_columns (optional): Number of columns in the figure. Default is 2.
    - height (optional): Height of each subplot in pixels. Default is 150.
    - y_order (optional): List of values to specify the order of the y axis. Default is None.

    """
    if columns == None:
        columns = df.dtypes[df.dtypes != 'object'].index.tolist()
    n_rows = (len(columns)-1) // n_columns + 1
    fig = make_subplots(rows=n_rows, cols=n_columns, subplot_titles=columns)
    for i, feature in enumerate(columns):
        if classification:
            zero = df.sort_values(feature)[df.sort_values(feature)[classification] != label]
            one = df.sort_values(feature)[df.sort_values(feature)[classification] == label]
            fig.add_trace(go.Histogram(y=zero[groupby], x=zero[feature],
                histfunc=agg, 
                marker_color='blue',
                orientation='h',
                opacity=0.5), 
                row=i//n_columns+1, col=i % n_columns + 1
                )
            fig.add_trace(go.Histogram(y=one[groupby], x=one[feature],
                histfunc=agg, 
                marker_color='red',
                orientation='h',
                opacity=0.5),
                row=i//n_columns+1, col=i % n_columns + 1)
        else:
            fig.add_trace(go.Histogram(
                histfunc=agg, x=df[feature], y=df[groupby],
                orientation='h'
                ), 
            row=i//n_columns+1, col=i % n_columns + 1)
    
    if classification:
        title = f'Observations with {classification} of value {label} are indicated in red'
    else:
        title = 'Value counts'
    fig.update_layout(
        showlegend=False,
        height=(n_rows+1)*height,
        barmode=barmode,
        # bargap=0.1,
        title=title,
        title_x=0.5,
        title_xanchor='center',
        # title_y=0.1,
        title_yanchor='top'
    )
    fig.update_xaxes(title=dict(
        standoff=0,
        ),
        title_text=f'{agg}'.upper(),
        row=n_rows
    )
    if y_order:
        aggregate_df = df[columns + [groupby]].groupby(
            groupby
            ).sum().sort_values(by=[columns[0]], ascending=True)
        index_list = aggregate_df.index.tolist()
        print(f'y_order: {index_list}')
        fig.update_yaxes(categoryorder='array', categoryarray=index_list)
    fig.show()


def plot_by_category(
    df, category_column='Category', classification='Unbilled', label=True,
    agg='sum',
    sort_column='Elapsed'
    ):
    classified_df = classify_projects(df)
    aggregate_df = classified_df[[sort_column, category_column]].groupby(
        category_column
        ).sum().sort_values(by=[sort_column], ascending=True)
    # index_list = aggregate_df.index.tolist()
    # print(index_list)

    plot_int_hist(
        classified_df, 
        groupby=category_column,
        columns=[sort_column],
        classification=classification,
        agg=agg,
        label=label,
        y_order=True
    )
    return aggregate_df

plot_by_category(df, classification='Unbilled')

There are 6 categories: ['tech career development', 'career positioning', 'Other', 'Ginkgo', 'personal training work', 'tech business']
y_order: ['personal training work', 'career positioning', 'Ginkgo', 'tech business', 'Other', 'tech career development']


,Elapsed
Category,
personal training work,31.666667
career positioning,44.083333
Ginkgo,99.900000
tech business,148.033333
Other,301.183333
tech career development,370.350000


# Update Classificaiton

In [59]:
from wrangling import *

def classify_unbilled(df, column='Flag', flag_name='do not bill'):

    def parse_flag(row, flag_name=flag_name):
        if row[column]:
            for flag in row[column]:
                if flag['name'] == flag_name:
                    return True
                else:
                    return False
        else:
            return False

    classified_df = df.copy()
    classified_df['Unbilled'] = classified_df.apply(parse_flag, axis=1)
    return classified_df

    
def classify_projects(df, column='Task Project name', tag_column='Task Project tags'):
    def classify(row):
        tag_set = set(row[tag_column]) if len(row[tag_column]) > 0 else {row[tag_column]}
        if ('ginkgo' in row[column].lower()) | ('ginkgo work' in [tag.lower() for tag in row[tag_column]]):
            row['Category'] = 'Ginkgo'
        elif row[column].lower() in ['ghl chatbot', 'tech business']:
            row['Category'] = 'tech business'
            if row[column].lower() == 'tech business':
                row['Unbilled'] = True
        elif tag_set.intersection({'portfolio', 'tech ed'}):
            row['Category'] = 'tech career development'
            row['Unbilled'] = True
        elif 'defy time fitness' in [tag.lower() for tag in row[tag_column]]:
            row['Category'] = 'personal training work'
            if 'defy time fitness' not in row[column].lower():
                row['Unbilled'] = True
        elif row[column].lower() == 'career positioning':
            row['Category'] = 'career positioning'
            row['Unbilled'] = True
        elif 'admin' in [tag.lower() for tag in row[tag_column]]:
            row['Category'] = 'admin'
            row['Unbilled'] = True
        elif 'volunteer' in [tag.lower() for tag in row[tag_column]]:
            row['Category'] = 'volunteer'
            row['Unbilled'] = True
        else:
            row['Category'] = 'Other'
            row['Unbilled'] = True

        return row
    
    classified_df = classify_unbilled(df).apply(lambda x: classify(x), axis=1)
    unique_categories = classified_df['Category'].unique()
    print(f'There are {len(unique_categories)} categories: {[category for category in unique_categories]}')
    return classified_df

classified_df = classify_projects(df)
filters = {
    'Category': ['other']
}
filter_df_all_conditions(classified_df, filters)

There are 8 categories: ['tech career development', 'career positioning', 'Ginkgo', 'admin', 'personal training work', 'tech business', 'volunteer', 'Other']
Results where ALL conditions are met:
	DataFrame indices: [560, 563, 634, 668, 680, 711, 732, 748, 803, 870, 879, 939, 940, 943, 944, 979, 980, 1250, 1251, 1294, 1295, 1296, 1297]
	DataFrame shape: (23, 17)


,Name,created_time,Task Tag,Flag,Elapsed,Task Name,Task Project tags,Task Parent-task,Task Sub-tasks,Task Project name,start time,end time,Roadmap Item,url,id,Unbilled,Category
560,2023-11-02 08:20,2023-11-02 08:20:00-07:00,[life],None,0.266667,counselling prep/homework,[life],[None],[None],Inner work,08:20,08:36,[],https://www.notion.so/2023-11-02-08-20-de259e2...,291,True,Other
563,2023-11-02 11:00,2023-11-02 11:01:00-07:00,[life],None,0.950000,counselling session,[life],[None],[None],Inner work,09:58,10:55,[],https://www.notion.so/2023-11-02-11-00-6483113...,288,True,Other
634,2023-11-09 13:14,2023-11-09 13:14:00-08:00,[life],None,1.233333,counselling session,[life],[None],[None],Inner work,12:00,13:14,[],https://www.notion.so/2023-11-09-13-14-3aabcf3...,217,True,Other
668,2023-11-12 23:45,2023-11-12 23:45:00-08:00,[life],None,0.166667,weekly/monthly reflection,[life],[None],[None],Inner work,23:00,23:10,[],https://www.notion.so/2023-11-12-23-45-be38075...,183,True,Other
680,2023-11-13,2023-11-13 21:50:00-08:00,[life],None,0.100000,Communication with Optio,[life],[None],[None],Optio,21:44,21:50,[],https://www.notion.so/2023-11-13-2138212ed8f64...,171,True,Other
711,2023-11-17 13:52,2023-11-17 13:52:00-08:00,[life],None,1.000000,Communication with Optio,[life],[None],[None],Optio,14:00,15:00,[],https://www.notion.so/2023-11-17-13-52-2cd8e46...,140,True,Other
732,2023-11-19 22:45,2023-11-19 22:45:00-08:00,[life],None,0.583333,weekly/monthly reflection,[life],[None],[None],Inner work,22:45,23:20,[],https://www.notion.so/2023-11-19-22-45-8885a18...,119,True,Other
748,2023-11-21 10:37,2023-11-21 10:37:00-08:00,[life],None,0.350000,Communication with Optio,[life],[None],[None],Optio,10:16,10:37,[],https://www.notion.so/2023-11-21-10-37-885ca6f...,103,True,Other
803,2023-11-26 21:36,2023-11-26 21:36:00-08:00,[life],None,0.350000,weekly/monthly reflection,[life],[None],[None],Inner work,21:36,21:57,[],https://www.notion.so/2023-11-26-21-36-b4bf658...,48,True,Other
870,2023-12-04 10:00,2023-12-04 10:10:00-08:00,[life],None,0.333333,weekly/monthly reflection,[life],[None],[None],Inner work,10:00,10:20,[],https://www.notion.so/2023-12-04-10-00-6589372...,18,True,Other


### plot

In [61]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_int_hist(
    df, groupby, columns=None, classification=None, label=1, 
    agg='sum',
    barmode='stack', n_columns=1, height=150, y_order=False
    ):
    """
    Use Plotly to plot multiple histograms using the specified columns of a dataframe.
    https://plotly.com/python/histograms/#specify-aggregation-function

    Arguments:
    - df: Dataframe.
    - columns (optional): Columns of dataframe on which to create the histogram. If blank, all numeric data will be plotted.
    - classification (optional): Provide name of colum containing binary classification values 0 and 1. 
        Data points classified as 1 will be in red.
    - label (optional): Label of classification column. Default is 1.
    - barmode ('stack', 'group', or 'overlay'; optional): How the different will be shown. Default is 'stack'.
    - n_columns (optional): Number of columns in the figure. Default is 2.
    - height (optional): Height of each subplot in pixels. Default is 150.
    - y_order (optional): List of values to specify the order of the y axis. Default is None.

    """
    if columns == None:
        columns = df.dtypes[df.dtypes != 'object'].index.tolist()
    n_rows = (len(columns)-1) // n_columns + 1
    fig = make_subplots(rows=n_rows, cols=n_columns, subplot_titles=columns)
    for i, feature in enumerate(columns):
        if classification:
            zero = df.sort_values(feature)[df.sort_values(feature)[classification] != label]
            one = df.sort_values(feature)[df.sort_values(feature)[classification] == label]
            fig.add_trace(go.Histogram(y=zero[groupby], x=zero[feature],
                histfunc=agg, 
                marker_color='blue',
                orientation='h',
                opacity=0.5), 
                row=i//n_columns+1, col=i % n_columns + 1
                )
            fig.add_trace(go.Histogram(y=one[groupby], x=one[feature],
                histfunc=agg, 
                marker_color='red',
                orientation='h',
                opacity=0.5),
                row=i//n_columns+1, col=i % n_columns + 1)
        else:
            fig.add_trace(go.Histogram(
                histfunc=agg, x=df[feature], y=df[groupby],
                orientation='h'
                ), 
            row=i//n_columns+1, col=i % n_columns + 1)
    
    if classification:
        title = f'Observations with {classification} of value {label} are indicated in red'
    else:
        title = 'Value counts'
    fig.update_layout(
        showlegend=False,
        height=(n_rows+1)*height if height else None,
        barmode=barmode,
        # bargap=0.1,
        title=title,
        title_x=0.5,
        title_xanchor='center',
        # title_y=0.1,
        title_yanchor='top'
    )
    fig.update_xaxes(title=dict(
        standoff=0,
        ),
        title_text=f'{agg}'.upper(),
        row=n_rows
    )
    if y_order:
        aggregate_df = df[columns + [groupby]].groupby(
            groupby
            ).sum().sort_values(by=[columns[0]], ascending=True)
        index_list = aggregate_df.index.tolist()
        print(f'y_order: {index_list}')
        fig.update_yaxes(categoryorder='array', categoryarray=index_list)
    fig.show()


def plot_by_category(
    df, category_column='Category', classification='Unbilled', label=True,
    agg='sum', sort_column='Elapsed', height=200
    ):
    classified_df = classify_projects(df)
    aggregate_df = classified_df[[sort_column, category_column]].groupby(
        category_column
        ).sum().sort_values(by=[sort_column], ascending=True)
    # index_list = aggregate_df.index.tolist()
    # print(index_list)

    plot_int_hist(
        classified_df, 
        groupby=category_column,
        columns=[sort_column],
        classification=classification,
        agg=agg,
        label=label,
        y_order=True,
        height=height
    )
    return aggregate_df

plot_by_category(df, classification='Unbilled')

There are 8 categories: ['tech career development', 'career positioning', 'Ginkgo', 'admin', 'personal training work', 'tech business', 'volunteer', 'Other']
y_order: ['volunteer', 'Other', 'personal training work', 'career positioning', 'admin', 'tech business', 'Ginkgo', 'tech career development']


,Elapsed
Category,
volunteer,4.450000
Other,11.900000
personal training work,31.666667
career positioning,44.083333
admin,51.933333
tech business,148.033333
Ginkgo,332.800000
tech career development,370.350000


# *End of Page*